In [5]:
#!/usr/bin/env python


## parameters 
from options import get_options

opts = get_options()
opts

Namespace(problem='tsp', graph_size=20, batch_size=512, epoch_size=1280000, val_size=10000, val_dataset=None, model='attention', embedding_dim=128, hidden_dim=128, n_encode_layers=3, tanh_clipping=10.0, normalization='batch', lr_model=0.0001, lr_critic=0.0001, lr_decay=1.0, eval_only=False, n_epochs=100, seed=1234, max_grad_norm=1.0, no_cuda=False, exp_beta=0.8, baseline=None, bl_alpha=0.05, bl_warmup_epochs=0, eval_batch_size=1024, checkpoint_encoder=False, shrink_size=None, data_distribution=None, log_step=50, log_dir='logs', run_name='run_20231215T113614', output_dir='outputs', epoch_start=0, checkpoint_epochs=1, load_path=None, resume=None, no_tensorboard=False, no_progress_bar=False, use_cuda=True, save_dir='outputs/tsp_20/run_20231215T113614')

In [11]:
## generate and load data
import numpy as np
def generate_tsp_data(dataset_size, tsp_size):
    return np.random.uniform(size=(dataset_size, tsp_size, 2)).tolist()

dataset_size = 10000
graph_size = 20

dataset = generate_tsp_data(dataset_size, graph_size)



In [ ]:
## graph embedding

import torch.nn as nn 
from graph_encoder import GraphAttentionEncoder
node_dim = 2  # x, y
embedding_dim = 128
init_embed = nn.Linear(node_dim, embedding_dim)

# embedder = GraphAttentionEncoder(
#             n_heads=n_heads,
#             embed_dim=embedding_dim,
#             n_layers=self.n_encode_layers,
#             normalization=normalization
#         )

# embeddings, _ = embedder(init_embed(input))

In [ ]:
from graph_encoder import GraphAttentionEncoder

import torch
import torch.nn as nn
from options import get_options


class AttentionModel(nn.Module):
    def __init__(self,
                 embedding_dim,
                 hidden_dim,
                 problem,
                 n_encode_layers=2,
                 tanh_clipping=10.,
                 mask_inner=True,
                 mask_logits=True,
                 normalization='batch',
                 n_heads=8,
                 checkpoint_encoder=False,
                 shrink_size=None):
        super(AttentionModel, self).__init__()
        node_dim = 2  # x, y
        self.embedder = GraphAttentionEncoder(
            n_heads=n_heads,
            embed_dim=embedding_dim,
            n_layers=self.n_encode_layers,
            normalization=normalization
        )

        self.init_embed = nn.Linear(node_dim, embedding_dim)


    def _init_embed(self, input):

        if self.is_vrp or self.is_orienteering or self.is_pctsp:
            if self.is_vrp:
                features = ('demand', )
            elif self.is_orienteering:
                features = ('prize', )
            else:
                assert self.is_pctsp
                features = ('deterministic_prize', 'penalty')
            return torch.cat(
                (
                    self.init_embed_depot(input['depot'])[:, None, :],
                    self.init_embed(torch.cat((
                        input['loc'],
                        *(input[feat][:, :, None] for feat in features)
                    ), -1))
                ),
                1
            )
        # TSP
        return self.init_embed(input)